In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
aulas = pd.read_excel('/home/cayena/Downloads/planilha_teste_thefamily.xlsx')

In [3]:
aulas['Horário'] = aulas['Horário'].astype(str)

In [4]:
aulas['segunda'] = aulas['Dias da Semana'].str.contains('2ª').astype(int)
aulas['terça'] = aulas['Dias da Semana'].str.contains('3ª').astype(int)
aulas['quarta'] = aulas['Dias da Semana'].str.contains('4ª').astype(int)
aulas['quinta'] = aulas['Dias da Semana'].str.contains('5ª').astype(int)

In [5]:
df_horarios = aulas['Horário'].str.get_dummies()
df_unidade = aulas['Unidade'].str.get_dummies()

In [6]:
aulas_geral = pd.concat([aulas, df_horarios,df_unidade], axis=1)

In [7]:
grupo_semana = aulas_geral[['Grupo', 'segunda', 'terça', 'quarta', 'quinta']].copy()
grupo_horario = aulas_geral.drop(columns=[ 'Horário', 'Unidade', 'Dias da Semana','segunda', 'terça', 'quarta', 'quinta', 'Jardim', 'Satélite', 'Vicentina']).copy()
grupo_loc = aulas_geral[['Grupo', 'Jardim','Satélite','Vicentina']].copy()

In [8]:
Professores = {'Name':['Allan','Carlos','Henrique','Yasmin','Malu','Tanze','Okamoto','Edu','Wolf','Odin','Renan','Gabiraes','Nicolas','Novaes','Jorge','Kaori']}
profes = pd.DataFrame(Professores)
profes.shape

(16, 1)

In [9]:
# sortear uma aula para cada professor
# usar a função de minimizar para completar os horarios e turmas restantes

In [10]:
aulas_segunda = aulas_geral[aulas_geral['segunda'] == 1].copy()

In [11]:
from ortools.sat.python import cp_model

In [16]:
all_teacher = profes['Name'].to_list()
all_time = aulas_segunda['Horário'].to_list()

model = cp_model.CpModel()

shifts = {}
for i in all_teacher:
    for j in all_time:
        shifts[(i, j)] = model.NewBoolVar(f"{i}_converinglesson_{j}")

for j in all_time:
    model.Add(sum(shifts[(i, j)] for i in all_teacher) == 1)

solver = cp_model.CpSolver()
status = solver.Solve(model)

In [18]:
if status == cp_model.FEASIBLE:
    print("Solução encontrada!")
    
    # Iterando sobre os professores e horários e exibindo as alocações
    for j in all_time:
        for i in all_teacher:
            if solver.Value(shifts[(i, j)]) == 1:
                print(f"Professor {i} foi alocado para o horário {j}")
else:
    print("Nenhuma solução viável encontrada.")

Nenhuma solução viável encontrada.


In [15]:
aulas_segunda['Horário'].value_counts()

19:00:00    16
18:00:00     9
20:00:00     6
08:00:00     5
16:00:00     4
14:00:00     2
09:00:00     2
17:00:00     2
07:00:00     1
12:00:00     1
20:30:00     1
15:00:00     1
Name: Horário, dtype: int64